<center><H1>Un primer estudio para la detección de anomalías</H1><center>

<center><img src="https://www.gstatic.com/devrel-devsite/prod/ve2848ad92313fddfcd40baeb58a2f663fe2fd55c371a714a6bb3e329e2b15223/tensorflow/images/lockup.svg"  height="80px" style="padding-bottom:5px;"  /></center>

<center><H2>Julio Waissman Vilanova</H2>

<table align="center">
      <td align="center"><a target="_blank" href="https://www.unison.mx">
            <img src="https://www.unison.mx/wp-content/themes/awaken/images/logo.png"  height="70px" style="padding-bottom:5px;"  /></a></td>  
      <td align="center"><a target="_blank" href="https://www.gob.mx/cenace">
            <img src="https://universidad.cenace.gob.mx/pluginfile.php/244/block_html/content/CENACE-logo-completo.png" width="300" style="padding-bottom:5px;" /></a></td>
      <td align="center"><a target="_blank" href="https://colab.research.google.com/github/juliowaissman/rn-cenace/blob/main/anomalias.ipynb">
            <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a></td>

</table>

In [12]:
# Las bibliotecas de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Para normalizar los datos de entrada
from sklearn.preprocessing import MinMaxScaler

#Tensorflow con keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Gráficas más fáciles de manipular con plotly
import plotly.express as px
import plotly.graph_objects as go

# Como se verán las gráficas de matplotlib
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (30,15)

In [ ]:
url = "https://github.com/juliowaissman/rn-cenace/raw/main/proyectos/Alfredo/7Sep2021_2047hrs.xlsx"
df_raw1 = pd.read_excel(url, sheet_name=0)
print(df_raw1.info())
df_raw2 = pd.read_excel(url, sheet_name=1)
print(df_raw2.info())
df_raw3 = pd.read_excel(url, sheet_name=2)
print(df_raw3.info())


In [ ]:
df_raw1.set_index(df_raw1['Timestamp (UTC)'], inplace=True)

# Frecuencias raras: 9, 13, 14, 28, 30, 36, 
# Escalonados: 37, 38, 
# Posibles fallas: 18, 19
df_raw1.iloc[:, [i for i in range(1,36) if i not in [9, 13, 14, 18, 19, 28, 30]]].plot()


In [ ]:
a = df_raw1.iloc[:, [i for i in range(1,36) if i not in [9, 13, 14, 18, 19, 28, 30]]].values
df_raw1['Media'] = a.mean(axis=1)
df_raw1['Std'] = a.std(axis=1)
df_raw1['Q3inc'] = np.percentile(a, 75, axis=1) - df_raw1.Media
df_raw1.Std.plot()

In [ ]:
df_raw1.Media.diff(3).plot()

In [ ]:
fig = go.Figure([
    go.Scatter(
        x=df_raw1.index, y=df_raw1.Media,
        name='Media', mode='lines', line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        x=df_raw1.index, y=df_raw1.Media + df_raw1.Std,
        name='+std', mode='lines', marker=dict(color="#444"),
        line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        x=df_raw1.index, y=df_raw1.Media - df_raw1.Std,
        name='-std', marker=dict(color="#444"),
        line=dict(width=0), mode='lines', fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty', showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Frecuencia',
    title='Frecuencia media y desviación estandar',
    hovermode="x"
)
fig.show()